In [ ]:
!pip -q install datasets
!pip install -U sentence-transformers
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from sentence_transformers import SentenceTransformer, util

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 3.3 MB/s eta 0:00:00


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def load_phrases():
    dataset = load_dataset("PiC/phrase_similarity")
    dataset = dataset.map(encode_phrases, batched=True)
    return dataset

def load_sentences():
    dataset = load_dataset("paws", "labeled_final")
    dataset = dataset.map(encode_sent, batched=True)
    return dataset

def encode_phrases(examples):
    return tokenizer(examples["phrase1"], examples["phrase2"], truncation=True, padding="max_length")

def encode_sent(examples):
    return tokenizer(examples["sentence1"], examples["sentence2"], truncation=True, padding="max_length")


In [ ]:
def embed_and_cosine(sentence1, sentence2):
    embeddings1 = sim_model.encode(sentence1)
    embeddings2 = sim_model.encode(sentence2)

    cosine_scores = util.cos_sim(embeddings1, embeddings2)
    return cosine_scores

In [ ]:
sim_model = SentenceTransformer("all-MiniLM-L6-v2")

boundary_score = 0.5
dataset = load_phrases()
train_data = dataset["test"]

tp, tn, fp, fn = 0, 0, 0, 0

for item in train_data:
    ind = item["label"]
    phrase1 = item["phrase1"]
    phrase2 = item["phrase2"]
    score = embed_and_cosine(phrase1, phrase2)
    if (score >= boundary_score):
        if (ind):
            tp += 1
        else:
            fp += 1
    else:
        if (ind):
            fn += 1
        else:
            tn += 1

print((tp + tn)/(tp + tn + fp + fn))


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7004 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

0.5038549400342661
